# Qiskit Fall Fest @ Yonsei 2025 : Beginner Hackathon

> Qiskit Fall Fest @ Yonsei 2025 비기너 해커톤에 오신 모든 분들을 환영합니다! 🎉  
> 이 노트북은 양자 컴퓨팅과 Qiskit의 핵심 개념을 처음 해보시는 분들도 즐겁게 배우고 직접 코딩하며 익힐 수 있도록 제작되었습니다.  
> 끝까지 완료하시면 비기너 챌린지 경품 추첨에 자동으로 응모되니, 도전과 함께 행운의 기회도 잡아보세요!

**진행 방법:**

1.  **개념 학습**: 각 주제에 대한 설명을 꼼꼼히 읽어보세요.
2.  **코드 작성**: 설명 아래 셀에서 `TODO`라 써있는 부분에 문제에 맞는 코드를 작성합니다.
3.  **정답 확인**: 코드 작성이 끝나면, 바로 아래 `Grader Cell`을 실행해 정답 여부를 확인하세요.
4.  **경품 응모**: 마지막 문제까지 모두 통과하면 경품 추첨에 자동으로 응모됩니다.

### GOOD LUCK 🤞

## Setup

먼저 아래 셀을 실행해서 이번 챌린지에서 필요한 패키지들을 설치하고 불러옵니다

In [ ]:
! pip install numpy qiskit qiskit-aer qiskit-ibm-runtime matplotlib
! pip install requests

import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter, ParameterVector
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService

import qff25_yonsei_beginner_grader as grader

In [ ]:
# Initialize grader

user_name = # 동명이인이 있을 수 있으니, (이름_전화번호 4자리)를 입력하세요. ex)홍길동_3456
grader.initialize_grader(user_name)

---
## 1. Pauli 연산자 (단일 큐비트 연산자)

Pauli 연산자(X, Y, Z, I)는 기본적인 단일 큐비트(single-qubit) 양자 연산을 나타내는 2x2 행렬입니다.

Qiskit에서는 `Pauli` 클래스에 원하는 연산자의 문자열을 파라미터로 보내서 Pauli 연산자를 생성할 수 있습니다 (예: X 연산자는 `Pauli('X')`).

Pauli 연산자에는 X, Y, Z, I 게이트가 있습니다. 다음은 각 단일 큐비트 게이트에 대한 설명입니다.

* ### <h3><span style="color:orange;">항등 연산자 (I, Identity)</span></h3>

**"아무것도 하지 않는"** 연산입니다. 어떤 큐비트 상태에 적용하든 그 상태를 그대로 유지시킵니다.

$$ I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} $$

* ### <span style="color:orange;">X 연산자 (Pauli-X)</span>
고전 컴퓨터의 **NOT 게이트**와 가장 유사한 연산입니다. 큐비트의 상태를 정반대로 뒤집습니다.

* $|0\rangle$ 상태는 $|1\rangle$로 바뀝니다.
* $|1\rangle$ 상태는 $|0\rangle$으로 바뀝니다.
* Bloch Sphere의 **X축**을 기준으로 180도 회전시키는 것과 같습니다.

$$ X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} $$

* ### <span style="color:orange;">Z 연산자 (Pauli-Z)</span>
**"위상(Phase)을 뒤집는"** 연산입니다. 고전 컴퓨터에는 없는 독특한 양자적 특징을 보여줍니다.

* $|0\rangle$ 상태는 그대로 유지됩니다.
* $|1\rangle$ 상태의 부호(위상)를 뒤집어 $-|1\rangle$로 만듭니다.
* Bloch Sphere의 **Z축**을 기준으로 180도 회전시키는 것과 같습니다.

$$ Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} $$

* ### <span style="color:orange;">Y 연산자 (Pauli-Y)</span>
큐비트의 상태를 뒤집고(X 연산처럼) 동시에 위상도 바꾸는(Z 연산처럼) 복합적인 연산을 수행합니다.

* Bloch Sphere의 **Y축**을 기준으로 180도 회전시키는 것과 같습니다.

$$ Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} $$

---

#### 또한 각 큐비트에 문자를 지정하여 다중 큐비트(multi-qubit) 파울리 연산자도 만들 수 있습니다.
Qiskit은 **little-endian**  비트 순서를 따르기 때문에 `'XI'`는 0번 큐비트에 `'I'`를, 1번 큐비트에 `'X'` 연산을 적용하는 것을 의미합니다.

#### 💡 잠깐: 리틀 엔디안(Little-Endian)이란?

리틀 엔디안은 데이터의 가장 '작은 단위(little-end)'를 가장 먼저 기록하고 처리하는 방식입니다.
예를 들어 **'254'** 를 little-endian으로 쓰면 **'452'** 로 표현합니다.

따라서 `'XI'`의 경우, 문자열을 오른쪽에서 왼쪽으로 읽으며 큐비트 번호를 0, 1, 2... 순서로 대입하면 됩니다.

I (가장 오른쪽) → qubit 0

X (그 다음 왼쪽) → qubit 1


따라서 0번 큐빗에 `'I'`를 1번 큐빗에 `'X'`를 가하는 연산자는 `Pauli('XI')`로 만들 수 있습니다.

---

### **문제 1 :**
다음 기능을 수행하는 코드를 작성하세요:
1. `Pauli()`를 사용해 큐비트 0에 `I`(항등), 큐비트 1에 `Y`, 큐비트 2에 `Z` 연산을 적용하는 3큐비트 파울리 연산자를 생성합니다. (Qiskit의 little-endian 순서에 따라 문자열은 `'ZYI'`가 됩니다.)
2. `to_matrix()`를 사용해 연산자에 해당하는 행렬을 출력합니다.

In [ ]:
pauli_op = # TODO
matrix = pauli_op.to_matrix()

print(matrix)

In [ ]:
# Grader Cell
grader.grade_p1(pauli_op)

---

## 2. 단일 큐비트 게이트와 위상(Phase)

X, Y, Z, H, S, T와 같은 단일 큐비트 게이트는 하나의 큐비트에 적용되는 기본 연산입니다.
* ### <span style="color:orange;">H Gate (Hadamard Gate)</span>
Hadamard 게이트는 중첩(superposition)을 만드는 가장 대표적인 게이트입니다. 양자 컴퓨팅의 핵심적인 특징을 만들어내는 데 필수적인 역할을 합니다.

블로흐 구에서 Z축의 상태($|0\rangle, |1\rangle$)를 X축의 상태($|+\rangle, |-\rangle$)로 바꾸는 회전으로 생각할 수 있습니다.
* $|0\rangle$ 상태에 적용 시:
    $H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) = |+\rangle$
* $|1\rangle$ 상태에 적용 시:
    $H|1\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle) = |-\rangle$

H 게이트를 두 번 연속으로 적용하면 원래 상태로 돌아옵니다. ($H \cdot H = I$)
$$H = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$$
* ### <span style="color:orange;">S Gate (Phase Gate)</span>
S 게이트는 위상(Phase) 게이트라고도 불리며, 블로흐 구의 Z축을 중심으로 **90도(π/2)**만큼 위상을 회전시킵니다.
* ∣0⟩ 상태는 그대로 유지됩니다.
* ∣1⟩ 상태에 $e^iπ/2=i$ 만큼의 위상 변화를 줍니다. (∣1⟩→i∣1⟩)
$$S = \begin{pmatrix} 1 & 0 \\ 0 & i \end{pmatrix}$$

* ### <span style="color:orange;">T Gate (π/8 Gate)</span>
T 게이트는 Bloch Sphere에서 Z축을 중심으로 **45도($\pi/4$)** 만큼 위상을 회전시킵니다.
* $|0\rangle$ 상태는 그대로 유지됩니다.
* $|1\rangle$ 상태에 $e^{i\pi/4}$ 만큼의 위상 변화를 줍니다. ($|1\rangle \rightarrow e^{i\pi/4}|1\rangle$)
$$T = \begin{pmatrix} 1 & 0 \\ 0 & e^{i\pi/4} \end{pmatrix}$$

💡 핵심 관계: T 게이트는 S 게이트의 절반이라고 생각할 수 있습니다. T 게이트를 두 번 연속으로 적용하면 S 게이트와 동일한 연산이 됩니다. ($T^2 = S$)

### **문제 2 :**
다음 기능을 수행하는 코드를 작성하세요:
1. 큐비트 하나를 가지는 양자 회로를 생성합니다.
2. 해당 큐비트를 $|1\rangle$ 상태로 만듭니다.
3. 회로에 T 게이트를 추가하여 큐비트에 $\pi/4$ 만큼의 위상 변화를 적용합니다.

<details>
<summary style="font-size: large; font-weight: bold;">Hint</summary>
QuantumCircuit(num_qubits)에 회로의 큐빗 갯수를 파라미터로 보내서 양자 회로 객체를 만들 수 있습니다.

QuantumCircuit() object를 새로 만들면 각 wire의 state들은 $|0\rangle$로 초기화되어있습니다.

`'X'`게이트를 사용해서 $|0\rangle$을 $|1\rangle$로 바꿀 수 있습니다.
</details>

In [ ]:
qc = QuantumCircuit(1)

# TODO

statevector = Statevector(qc)
statevector.draw("latex")

In [ ]:
# Grader Cell
grader.grade_p2(qc)

---

## 3. Rotational Gates

`RX`, `RY`, `RZ`은 Rotational (X,Y,Z) 게이트로 각 게이트들은 Bloch Sphere의 각 축을 중심으로 회전을 수행합니다.

Rotational Gates를 활용해 **중첩(superposition)** 상태를 만들 수 있습니다.

* ### <span style="color:orange;">RX Gate</span>
X축을 중심으로 $\theta$만큼 회전시킵니다.
$$R_X(\theta) = \begin{pmatrix} \cos(\frac{\theta}{2}) & -i\sin(\frac{\theta}{2}) \\ -i\sin(\frac{\theta}{2}) & \cos(\frac{\theta}{2}) \end{pmatrix}$$

* ### <span style="color:orange;">RY Gate</span>
Y축을 중심으로 $\theta$만큼 회전시킵니다.

$$R_Y(\theta) = \begin{pmatrix} \cos(\frac{\theta}{2}) & -\sin(\frac{\theta}{2}) \\ \sin(\frac{\theta}{2}) & \cos(\frac{\theta}{2}) \end{pmatrix}$$

* ### <span style="color:orange;">RZ Gate</span>
Z축을 중심으로 $\theta$만큼 회전시킵니다.
$$R_Z(\theta) = \begin{pmatrix} e^{-i\theta/2} & 0 \\ 0 & e^{i\theta/2} \end{pmatrix}$$

예를 들어, 초기 상태 $|0\rangle$에 `RY(θ)`을 가하면, Y축을 중심으로 $\theta$ 각도만큼 회전해서 $cos(\frac\theta2)|0\rangle + sin(\frac\theta2)|1\rangle$와 같은 중첩 상태가 만들어집니다.

이때 0 또는 1을 측정할 확률은 각각의 진폭(amplitude)을 제곱한 값, 즉 $|cos(\frac\theta2)|^2$과 $|sin(\frac\theta2)|^2$이 됩니다.

### **문제 3 :**
다음 기능을 수행하는 코드를 작성하세요:
1. 큐비트 하나를 가지는 양자 회로를 생성합니다.
2. 초기 상태가 $|0\rangle$인 0번 큐비트에 단일 게이트를 적용하여, $|0\rangle$을 측정할 확률이 25%, $|1\rangle$을 측정할 확률이 75%가 되는 중첩 상태를 만듭니다.

<details>
<summary style="font-size: large; font-weight: bold;">Hint</summary>

`RY`를 사용하세요.

Roataional Gate를 사용할때는 첫번째 인자에 rotate하고 싶은 각도, 두번째 인자에 게이트를 가할 큐빗을 전달하면 됩니다.

</details>

In [ ]:
qc = # TODO

# TODO

sv = Statevector(qc)
probs = sv.probabilities_dict()
print(f'Probabilities: {probs}')
plot_bloch_multivector(sv)

In [ ]:
# Grader Cell
grader.grade_p3(qc)

---

## 4. 다중 큐비트 연산과 얽힘



CNOT(`qc.cx(control, target)`)과 같은 다중 큐비트 게이트는 중첩 상태에 있는 큐비트에 적용될 때 **얽힘(entanglement)** 상태를 만듭니다.

가장 대표적인 얽힘 상태인 Bell state $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$는 한 큐비트에 `H` Gate를 적용한 뒤 CNOT 게이트를 적용하여 만들 수 있습니다.

이때 Qiskit의 리틀 엔디안(little-endian) 비트 순서에 따라 큐비트 0이 가장 오른쪽 비트에 해당한다는 점을 기억하세요.

### **문제 4 :**
다음 기능을 수행하는 코드를 작성하세요:
1. 큐빗 두 개를 가지는 양자 회로를 생성합니다.
2. 첫 번째 큐빗(q0)을 제어 큐비트(control qubit)로 사용하여 Bell state $|\Phi^+\rangle$를 만듭니다.
3. 생성된 양자 회로를 `matplotlib`을 이용해 시각화하여 그립니다.
4. 회로의 최종 상태벡터를 출력합니다.

In [ ]:
qc = # TODO

# TODO

display(qc.draw('mpl'))
statevector = Statevector(qc)
print(statevector)

In [ ]:
# Grader Cell
grader.grade_p4(qc)

---

## 5. 양자 회로 만들기와 시각화 (Building Quantum Circuits and Drawing)

`QuantumCircuit` 클래스는 양자 회로를 만드는 데 사용됩니다. `.draw()` 메소드는 회로를 시각화하는 기능을 제공하며, `output='text', 'mpl', 'latex'`으로 다양한 출력 형식을 지정할 수 있습니다.

또한 `reverse_bits=True`를 통해 큐비트의 순서를 뒤집는 등 시각화 결과를 원하는 대로 조정할 수 있습니다.

`QuantumCircuit(num_qubit, num_classical)`로 회로의 큐빗 갯수와 고전 비트(classical bit) 갯수를 지정할 수 있습니다.

### **문제 5 :**
다음 기능을 수행하는 코드를 작성하세요:
1. 3큐빗 GHZ State를 만듭니다.
2. 큐빗 순서를 뒤집어 (q2가 위, q0가 아래에 표시되도록) `mpl`으로 회로를 그립니다.


<details>
<summary style="font-size: large; font-weight: bold;">Hint</summary>

<span style="font-size: large; font-weight:bold; color:orange;">GHZ State</span>

GHZ 상태는 세 개 이상의 큐비트가 얽혀있는 가장 대표적인 양자 상태입니다.

아래는 3-qubit GHZ 상태입니다.
$$|GHZ\rangle=\frac1{\sqrt2}(|000\rangle+|111\rangle)$$
GHZ State는 다음과 같은 방법으로 만들 수 있습니다.

1. 첫 번째 큐빗에 `H`를 가해 중첩상태를 만듭니다.

2. 첫 번째 큐빗을 control로 해서 나머지 큐빗에 순차적으로 `CNOT`을 가합니다.

</details>

In [ ]:
qc = # TODO

# TODO

In [ ]:
# Grader Cell
grader.grade_p5(qc)

---

## 6. 동적 회로와 고전적 제어 흐름 (Dynamic Circuits and Classical Control Flow)

Qiskit은 **동적 회로(dynamic circuits)** 를 지원하며, 이는 고전적인 측정 결과에 따라 다음 연산을 조건부로 실행할 수 있는 회로를 의미합니다.


`.if_test()` context manager를 사용하여 고전 비트(classical bit)의 값에 따라 특정 연산을 실행하는 조건부 블록을 만들 수 있습니다.

이를 통해 양자 프로그램 내에서 강력한 **고전 피드-포워드(classical feed-forward)** 를 구현할 수 있습니다.

### **문제 6 :**
다음 기능을 수행하는 코드를 작성하세요:
1. 큐비트 두 개와 고전 비트 한개를 가지는 양자 회로를 생성합니다.
2. 최하위 큐비트(qubit 0)에 Hadamard 게이트를 추가합니다.
3. 0번 큐빗을 측정한 결과를 0번 classical bit에 저장합니다.
4. 0번 큐비트를 측정한 결과가 `1`일 경우에만 1번 큐비트에 X 게이트를 적용합니다. 이 조건부 연산을 위해 `.if_test()` 컨텍스트 매니저를 사용하세요.
5. 완성된 회로를 `mpl`을 이용해 그립니다.

<details>
<summary style="font-size: large; font-weight: bold;">Hint</summary>

<span style="font-size: large; font-weight:bold; color:orange;">측정</span>

`qc.measure()` 메서드를 사용해 양자 상태를 측정할 수 있습니다.

`measure(q1, c1)`을 통해 양자 회로의 어떤 wire를 측정해서 결과를 어느 classical bit에 저장할지 지정할 수 있습니다.

`measure(q1, c1)`은 q1 큐빗을 측정해서 c1 고전 비트에 저장하라는 뜻입니다.

</details>

In [ ]:
qc = # TODO

# TODO

with qc.if_test((qc.clbits[0], 1)):
    # TODO

# TOCO

In [ ]:
# Grader Cell
grader.grade_p6(qc)

---

## 7. 양자 상태와 결과 시각화 (Visualizing Quantum States and Results)

Qiskit은 결과를 시각화하는 여러 함수를 제공합니다. `plot_histogram(counts)`은 시뮬레이션이나 실제 장치에서 실행한 측정 결과를 표시하는 데 사용됩니다.

예를 들어, 결과의 빈도순으로 정렬하여 분석을 더 쉽게 할 수 있습니다.



### **문제 7:**
다음 기능을 수행하는 코드를 작성하세요:
1. $|\Phi^+\rangle$ 벨 상태(Bell state)를 만드는 2개의 큐빗과 2개의 고전 비트를 갖고 있는 양자 회로를 생성합니다.
2. 0번과 1번 큐빗의 측정 결과를 각각 0번과 1번 고전 비트(classical bit)에 저장합니다.
3. `AerSimulator`를 사용하여 회로를 실행합니다.
4. 측정 결과(counts)를 얻습니다.
5. 측정 결과를 히스토그램으로 그립니다.

In [ ]:
bell = # TODO
# TODO


backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_bell = pm.run(bell)

sampler = Sampler(mode=backend)

job = sampler.run([isa_bell], shots=1024)
result = job.result()
counts = result[0].data.c.get_counts()

# TODO

In [ ]:
# Grader Cell
grader.grade_p7(counts)

---

## 8. Parameterized Quantum Circuits

Qiskit에서는 `Parameter` 클래스를 사용하여 symbolic parameter를 가지는 회로를 만들 수 있습니다.

이 파라미터들은 나중에 `.assign_parameters()` 메소드를 통해 특정 숫자 값으로 지정(bind)할 수 있는 일종의 placeholder 역할을 합니다.

이 기능은 VQE나 QAOA와 같은 변분 알고리즘(variational algorithm)의 핵심입니다.

### **문제 8 :**
다음 기능을 수행하는 코드를 작성하세요:
1. `theta`라는 이름의 파라미터를 나타내는 `Parameter` 인스턴스를 생성합니다.
2. 큐비트 하나를 가지는 `qc`라는 양자 회로를 생성합니다.
3. 해당 큐비트에 `theta` 파라미터를 사용하는 RX 게이트를 추가합니다.
4. `assign_parameters()`를 사용해 `theta` 파라미터를 `π/2` 값으로 지정(binding)하여 `bound_qc`라는 새로운 회로를 만듭니다.

<details>
<summary style="font-size: large; font-weight: bold;">Hint</summary>

<span style="font-size: large; font-weight:bold; color:orange;">Parameter에 값 할당하기</span>

`assign_parameters()`메서드로 값을 할당할때는 `{Parameter 이름 : 값}`으로 `dict`형태로 인자를 전달해야합니다.   

</details>

In [ ]:
theta = # TODO
qc = # TODO
# TODO

print("Original Unbound Circuit:")
display(qc.draw('mpl'))

bound_qc = # TODO

print('New Bound Circuit:')
bound_qc.draw('mpl')

In [ ]:
# Grader Cell
grader.grade_p8(qc, bound_qc)

---

## 9. 회로 트랜스파일링과 최적화 (Circuit Transpilation and Optimization)

**트랜스파일링(Transpilation)**은 양자 회로를 특정 양자 장치의 제약 조건, 즉 **기본 게이트(basis gates) 집합**과 **큐비트 연결성(connectivity)**에 맞게 변환하는 과정입니다.

고전 컴퓨터의 compiler와 비슷한 역할을 하는 단계입니다.

`generate_preset_pass_manager()`를 활용해 회로를 백엔드에 맞게 쉽게 transpiling할 수 있습니다.

`generate_preset_pass_manager()` 함수는 미리 설정된 구성으로 트랜스파일링 패스 매니저(pass manager)를 생성합니다.

이 함수는 여러 `optimization_level` 설정(0-3)을 가지고 있으며, 레벨이 높을수록 컴파일 시간은 길어지지만 더 발전된 최적화 기술을 적용하여 회로의 깊이(depth)와 게이트 수를 줄입니다.

* ### <span style="color:orange;">PassManager 사용법</span>
1. `generate_pass_manager()`를 사용해 `PassManager`객체를 만듭니다. 이때 원하는 backend를 `backend=`로 넘겨주고, `optimization_level=`로 최적화 정도를 지정해줍니다.
2. `pm.run()`메소드에 돌리고 싶은 회로를 전달해주면 transpile된 회로가 backend에서 실행됩니다.

#### 참고 : `generate_preset_pass_manager()`의 `optimization_level` 단계별 설명
* level=0 (최소): 거의 최적화를 수행하지 않습니다. 단순히 게이트를 기본 게이트로 변환하고 큐비트를 매핑하는 작업만 합니다. 컴파일 속도가 가장 빠릅니다.

* level=1 (가벼움): 가벼운 수준의 최적화를 수행합니다. 불필요한 게이트를 제거하는 등의 간단한 최적화 패스가 포함됩니다. (기본값)

* level=2 (무거움): 더 적극적인 최적화를 수행합니다. 게이트를 재배치하고 더 나은 큐비트 매핑을 찾는 등 더 복잡한 전략을 사용합니다. 컴파일 시간이 더 오래 걸립니다.

* level=3 (최대): 가능한 가장 강력한 최적화를 수행합니다. 최상의 회로를 찾기 위해 다양한 전략을 시도하며, 컴파일 시간이 매우 길어질 수 있습니다. 노이즈가 많은 실제 하드웨어에서 실행할 최종 회로처럼, 게이트 하나하나를 줄이는 것이 매우 중요할 때 사용합니다.

### **문제 9 :**
다음 기능을 수행하는 코드를 작성하세요:
1. 3큐비트 GHZ 회로를 생성합니다.
2. `optimization_level=3`으로 `FakeVigoV2` 백엔드에 맞게 회로를 트랜스파일링합니다.
3. 원본 회로의 깊이(depth)를 출력합니다.
4. 트랜스파일링된 회로의 깊이를 출력합니다.

In [ ]:
qc = # TODO
# TODO

backend = FakeVigoV2()
pm = # TODO

qc_isa = pm.run(qc)

print(f'Original circuit depth: {qc.depth()}')
print(f'Transpiled circuit depth: {qc_isa.depth()}')

In [ ]:
# Grader Cell
grader.grade_p9(qc_isa)

---

## 10. Qiskit Runtime 실행 모드 (Execution Modes)

**설명:** Qiskit Runtime은 **job**, **session**, **batch**의 세 가지 실행 모드를 제공합니다.

실행 모드는 작업(job)의 스케줄링 방식을 결정하며, 올바른 실행 모드를 선택하면 예산 내에서 워크로드를 효율적으로 실행할 수 있습니다.

#### 이번 주제에는 문제가 없습니다 😃

---

## 11. Quantum Primitives (Sampler and Estimator)

Quantum Primitives는 양자 컴퓨터(또는 시뮬레이터)와 상호작용하는 표준화된 인터페이스입니다. 주요 Primitives는 `Sampler`와 `Estimator` 두 가지입니다.

##### `Sampler`: 확률 분포 생성기

`Sampler`는 "이 양자 회로를 실행했을 때, 최종 측정 결과들의 확률 분포는 어떻게 될까?"라는 질문에 답합니다.

양자 회로를 실행하고, 각 측정 결과(비트스트링)가 몇 번이나 나왔는지 그 횟수(counts) 또는 확률을 반환합니다.

여러 개의 주사위를 던져 각 눈금이 몇 번 나왔는지 세는 통계 조사원과 비슷합니다.

##### `Estimator` : 기대값 계산기

`Estimator`는 "이 양자 회로가 만들어낸 양자 상태에 대해, 특정 Observable의 기대값(Expectation Value)은 얼마일까?"라는 질문에 답합니다.

양자 회로와 측정할 물리량(Observable)을 받아, 해당 물리량의 기대값을 계산하여 반환합니다.

따라서 `Estimator`는 양자회로 뿐만 아니라 Observable도 필요합니다.

`Sampler.run()` 또는 `Estimator.run()`에 transpile된 회로를 전달해 실행할 수 있습니다.

#### 이번 주제에는 문제가 없습니다 😃

#### 이전 문제의 답에서 Primitives를 어떻게 사용하는지 다시 한번 봐보세요!

---

## 12. Sampler Primitive 사용하기

Qiskit 2에서는 `qiskit_ibm_runtime`의 `Sampler` primitive를 `AerSimulator`와 같은 로컬 시뮬레이터와 함께 사용할 수 있습니다.

`Sampler`를 사용법 :

1. `Sampler(mode=backend)`로 Sampler를 initialize합니다.
2. `sampler.run([qc], shots=...)`로 Sampler Primitive를 실행합니다. 이때 전달하는 회로는 백엔드에 맞게 transpile되어 있어야하고, `list`에 담아 전달해야합니다.
3. `job.result()`를 통해 얻은 결과 객체에서, `result[0].data."claasical_register_이름"`을 통해 데이터에 접근할 수 있습니다.

### **문제 12:**
다음 기능을 수행하는 코드를 작성하세요:
1. $|\Phi^+\rangle$ 벨 상태(Bell state)를 만드는 양자 회로를 생성합니다.
2. `measure_all` 메소드를 사용하여 결과를 측정합니다.
3. `AerSimulator` 백엔드를 사용하여 회로를 트랜스파일링합니다. (`optimization_levle=1`로 헤주세요)
4. `Sampler`를 initialize합니다.
5. Sampler를 실행합니다. (`shots = 500`으로 해주세요)

In [ ]:
# TODO

backend = AerSimulator()
pm = # TODO
isa_bell = # TODO

# TODO
job = #TODO

result = job.result()
counts = result[0].data.meas.get_counts()
print(f'Measurement counts: {counts}')

In [ ]:
# Grader Cell
grader.grade_p12(counts)

---

## 13. Estimator Primitive 사용하기

Qiskit 2에서는 `qiskit_ibm_runtime`의 `Estimator` primitive를 `AerSimulator`와 같은 로컬 시뮬레이터와 함께 사용할 수 있습니다.

`Estimator` 사용법 :

1. `Estimator(mode=backend)`로 Estimator를 initialize합니다.
2. `apply_layout(isa_qc.layout)`으로 observable을 트랜스파일링한 회로(`isa_qc`)의 레이아웃으로 바꿉니다.
3.  `estimator.run([(circuit, observable)])` 메소드를 사용해서 `Estimator`를 실행합니다. 

    이때 전달하는 회로와 observable회로는 백엔드에 맞게 transpile되어 있어야하고, `(isa_qc, isa_observable)`형태의 튜플로 묶고, 이 튜플을 `list`에 담아 전달해야합니다.

4. `job.result()`로 얻은 결과 객체에서, 첫번째 객체인 `result[0]`의 `data.evs` attribute에 기대값이 있습니다.

### **문제 13 :**
다음 기능을 수행하는 코드를 작성하세요:
1. $|\Phi^+\rangle$ 벨 상태(Bell state)를 포함하는 양자 회로를 생성합니다.
2. Observable을 `SparsePauliOp('ZZ')`로 정의합니다.
3. `AerSimulator` 백엔드에 맞게 사용하여 Bell State 회로를 트랜스파일링합니다.
4. Observable 회로에 트랜스파일된 Bell State 회로 레이아웃을 적용합니다.
5. `Estimator` primitive를 초기화합니다.
6. `Estimator`를 실행합니다.
7. `job.result()`객체에서 기댓값을 가져옵니다.

In [ ]:
# TODO


exp_val = # TODO
print(f"Expectation value for ZZ: {exp_val}")

In [ ]:
# Grader Cell
grader.grade_p13(exp_val)

---

## 14. 오류 완화 기술 (Error Mitigation Techniques)

Qiskit은 양자 하드웨어의 노이즈(noise) 영향을 줄이기 위한 여러 기술을 제공합니다.

* **Readout Error Mitigation** : 최종 측정 단계에서 `0`을 `1`로 잘못 읽는 등의 오류를 보정합니다.
* **Dynamical Decoupling(DD)** : 큐비트가 유휴 상태일 때 펄스 시퀀스를 삽입하여 결맞음(decoherence)으로부터 큐비트를 보호합니다.
* **Zero-Noise Extrapolation(ZNE)** : 의도적으로 노이즈를 여러 단계로 증폭시켜 회로를 실행한 뒤, 그 결과를 노이즈가 0인 지점으로 외삽(extrapolate)하여 이상적인 결과를 예측하는 기법입니다.

#### 이번 주제에는 문제가 없습니다 😃

---

## 15. 실제 IBM 양자 하드웨어에서 실행하기

이전 문제에서는 `qiskit_ibm_runtime`의 `Sampler` primitive를 로컬 시뮬레이터인 `AerSimulator`와 함께 사용했습니다.

 이번에는 `Sampler` primitive를 실제 IBM 양자 컴퓨터에서 실행해 봅시다.

### **문제 19 :**

IBM Cloud에 접속해 실제 IBM 양자 컴퓨터를 사용하기 위새서는 IBM Cloud 계정을 만들고 API key를 받아야합니다.

1. https://quantum.cloud.ibm.com에 접속해 IBM 계정을 생성합니다.

2. IBM Quantum Platform에서 왼쪽 화면에 있는 API key 창에서 create으로 api key를 받습니다.

3. 생성된 API key를 복사해 아래 코드의 `your_api_key`에 넣습니다.

4. Create Instance를 눌러서 인스턴스를 만듭니다. 이때 Open Plan을 선택해서 만듭니다.

5. Instance를 만든 뒤 CRN을 복사해서 `your_crn`에 넣습니다.

In [ ]:
your_api_key = "EnterYourAPIHere"
your_crn = "EnterYourCRNHere"

QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
    name="qff25-ys",
    overwrite=True,
)

# Check that the account has been saved properly
service = QiskitRuntimeService(name="qff25-ys")
print(service.saved_accounts())

##### Bell state $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$를 5000번 sampling 한 통계를 실제 IBM 양자 컴퓨터로 확인해볼 것입니다.

아래 셀을 실행시켜 결과를 확인해보세요.

IBM Quantum Platform에서 workload에 들어가면 현재 내가 돌리고 있는 Job의 상태를 확인하실 수 있습니다.

In [ ]:
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

service = QiskitRuntimeService(name="qff25-ys")
backend = service.least_busy(operational=True, simulator=False)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_bell = pm.run(bell)

sampler = Sampler(mode=backend)

job = sampler.run([isa_bell], shots=5000)
result = job.result()

counts = result[0].data.meas.get_counts()
print(f"Measurement counts: {counts}")
plot_histogram(counts)

In [ ]:
# Grader Cell
grader.grade_p15(job)

### 결과 분석 :

Bell state $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$은 이론적으로는 절반은 $|00\rangle$, 절반은 $|11\rangle$이 나와야합니다.

하지만 실제 양자 컴퓨터에서는 다양한 오류, 잡음 등에 의해 이론과는 다르게 $|01\rangle$과 $|10\rangle$도 나오고, $|00\rangle$과 $|11\rangle$도 정확히 1:1이 아닙니다.

이와같이 현재 NISQ 하드웨어는 완벽하지 않으며 양자 컴퓨터는 갈 길이 멀었습니다.😢


---

## 16. Quantum Key Distribution (BB84)

키 분배(Key Distribution)란 암호화 통신에 사용할 비밀 키를 송신자와 수신자가 안전하게 공유하는 과정을 말합니다.
<br>
* ### <span style="color:orange;">상황 설정</span>
👩 Alice: 메시지를 보내는 사람 (Sender).

👨 Bob: 메시지를 받는 사람 (Receiver).

🕵️‍♀️ Eve: Alice와 Bob의 통신을 몰래 엿듣는 도청자 (Eavesdropper).

앨리스와 밥은 둘이 비밀 대화를 나누고 싶어서 대화를 암호화하기로 합니다. 그러려면 대화를 암호화 할 둘만 갖고 있는 '비밀 키'가 있어야 하죠.

하지만 Alice가 "Bob, 우리 비밀 키는 '토끼'로 하자!" 라고 메시지를 보내는 순간, 이들을 엿듣고 있는 도청자 Eve도 '토끼'라는 비밀 키를 알게 됩니다. 

결국 암호화는 소용없게 됩니다. 이처럼 안전하지 않은 경로를 통해 어떻게 비밀 키를 안전하게 공유할 것인가? 이것이 바로 키 분배 문제입니다.
* ### <span style="color:orange;">대칭키와 비대칭키</span>
이 '비밀 키'에는 크게 두가지 종류가 있습니다.
1. 대칭키

대칭키는 암호화할 때 사용하는 키와 암호를 풀 때(복호화) 사용하는 키가 서로 같은 방식입니다.

암호화 복호화 속도가 매우 빠르기 때문에 대용량 데이터를 암호화할 때 효율적이지만, 키를 전달하는 과정에서 Eve가 키를 몰래 복사해간다면, 이브도 비밀 대화를 복호화 할 수 있게 됩니다.

그래서 대칭키를 사용할 때는 맨 처음에 비밀 키를 어떻게 안전하게 전달할 것인가 하는 문제가 그대로 남습니다. 

2. 비대칭키

비대칭키는 대칭키의 키 분배 문제를 해결하기 위해 등장했습니다.

이름처럼 암호화하는 키와 복호화하는 키가 서로 다릅니다. 이 두 키는 수학적으로 짝을 이루며 각각을 '공개키'와 '개인키'라고 부릅니다.

- 공개키(Public Key) : 모든 사람에게 공개해도 됩니다. 이 키로는 암호화(잠그기)만 할 수 있습니다.
- 개인키(Private Key) : 자기 자신만 비밀스럽게 가지고 있어야 합니다. 이 키로만 암호를 풀 수 있습니다.

쉽게 설명하면 투입구만 있는 우체통을 생각하시면 됩니다.
1. Bob이 자신만 열 수 있는 특별한 우체통을 만들어서 동네 광장에 놓아둡니다. 이 우체통이 바로 밥의 공개키입니다.
2. Alice는 밥에게 보낼 편지를 이 우체통의 투입구에 넣습니다. (공개키로 암호화)
3. Eve를 포함한 동네 모든 사람이 이 우체통에 편지를 넣을 수는 있지만, 아무도 안의 편지를 꺼낼 수는 없습니다.
4. 오직 우체통의 주인인 Bob만이 자신의 개인키로 우체통을 열어 편지를 읽을 수 있습니다.
* ### <span style="color:orange;">Non Cloning Theorem</span>
Non cloning theroem에 의하면 임의의 모르는 양자 상태를 완벽하게 복사하여 똑같은 상태를 새로 만드는 것은 불가능합니다.

고전적인 비트(0과 1)은 쉽게 복사할 수 있지만, 중첩과 얽힘을 포함하는 큐비트는 그럴 수 없습니다.

큐비트의 모든 정보를 알아내려는 행위 자체가 일종의 '측정'이며, 이 과정에서 원본 큐비트의 상태가 필연적으로 변하거나 파괴되기 때문입니다.

따라서 이 양자역학적인 성질을 이용해 키 분배를 한다면, 비밀 키가 도청당했다면, Alice와 Bob은 키가 도청당했다는 사실을 반드시 알 수 있습니다.
* ### <span style="color:orange;">BB84</span>
BB84는 QKD의 가장 대표적인 프로토콜입니다.

BB84는 정보를 싣고 측정하는 방법으로 서로 다른 두개의 기저를 사용합니다.
- 사각 기저 (Rectangular Basis, Z) : 0을 $|0\rangle$, 1을 $|1\rangle$ 로 표현합니다.
- 대각 기저 (Diagonal Basis, X) : 0을 $|+\rangle$, 1을 $|-\rangle$ 로 표현합니다.

여기서 대각 기저의 $|+\rangle$ 와 $|-\rangle$ 은 Bell State $|+\rangle = \frac{1}{2}(|0\rangle + |1\rangle)$와 $|-\rangle = \frac{1}{2}(|0\rangle - |1\rangle)$ 을 의미합니다.

반대로 $|0\rangle$ 과 $|1\rangle$ 은 $|0\rangle = \frac{1}{2}(|+\rangle + |-\rangle)$와 $|1\rangle = \frac{1}{2}(|+\rangle - |-\rangle)$ 로 나타낼 수 있습니다.

따라서 만약 $|+\rangle$ 을 사각기저로 측정한다면, 50:50 확률로 0 또는 1이 나오고, $|0\rangle$ 을 대각기저로 측정한다면, 50:50 확률로 0 또는 1이 나오게 됩니다.

BB84는 바로 이 서로 다른 기저로 측정했을 때 측정 결과가 확률적으로 나온다는 점을 이용하는 프로토콜입니다.

##### BB84 프로토콜의 진행 단계는 다음과 같습니다 :
1. Alice의 인코딩 및 전송 : Alice는 보내고 싶은 비밀 키를 정하고 각 비트마다 사각 기저 또는 대각 기저중 하나를 무작위로 선택하여 큐비트를 인코딩한 후 Bob에게 보냅니다.

예를 들어 비밀키가 `01101`이고 Alice가 선택한 무작위 기저가 (+, +, +, x, x)이라면 양자상태 $|011+-\rangle$ 을 Bob에게 보낼 것입니다.

2. Bob의 측정 : Bob은 Alice가 어떤 기저를 무작위로 선택했는지 모르는 상태에서, 각 큐비트를 수신 받을 때마다, Bob도 사각 기저 또는 대각 기저 중 하나를 무작위로 선택하여 측정하고 결과를 기록합니다.

예를 들어 Bob이 (+, x, x, x, x) 기저로 Alice가 발신한 양자상태를 측정하기로 했다면, Bob이 Alice와 같은 기저를 선택한 1,4,5번째 큐빗은 

원래 Alice가 보내려했던 0,0,1이 100% 측정될 것이고, Alice와 다른 기저를 선택한 2,3번째 큐빗은 50:50의 확률로 0 또는 1을 측정하게 될 것입니다.

3. 기저 비교 : 이제 Alice와 Bob이 공개적인 채널(전화 등)을 통해 서로가 각 큐비트에 사용한 기저 목록만을 비교합니다. (이때 측정한 값은 절대 공개하지 않습니다)
    - 둘이 사용한 기저가 달랐던 경우의 결과는 의미가 없으므로 버립니다.
    - 둘이 사용한 기저가 같았던 경우의 결과만 남깁니다. 이 남겨진 비트들이 바로 **비밀 키 후보** 가 됩니다.

4. 도청자 확인 : Alice와 Bob은 남겨진 비밀 키 후보 중 일부를 무작위로 골라 공개적으로 비교합니다.
    - case 1) 만약 Eve가 도청했다면, non cloning theorem 때문에 반드시 측정을 해야만 했을 것입니다. 이때 Eve도 기저를 추측해 측정해야 하므로, Alice가 보낸 큐빗이 바뀔것 이기 때문에, <br>앨리스와 밥의 키 비교 과정에서 불일치가 나타날 것입니다.
    이 경우 비밀 키가 유출되었으니, 이 키를 페기하고 처음부터 다시 합니다.

    - case 2) 만약 불일치가 없다면, 도청자가 없다는 뜻이기 때문에 남은 키를 안전한 키로 확정하고 이를 비밀 키로 사용합니다.

### **문제 16 :**

이번 문제에서는 BB84를 직접 구현해볼것입니다.

도청자(Eve)가 있는 시나리오를 구현해볼 것입니다.

먼저 `encode_qubit()`은 Alice가 보내고 싶은 비밀키를 '+' 또는 'X'기저 중 하나로 인코딩하는 함수입니다.

그리고 `measure_qubit()`은 Bob이 수신받은 양자 상태를 '+' 또는 'X'기저 중 하나로 측정하는 함수입니다.

함수는 완성돼있으니 셀을 실행만 시켜주세요.

In [ ]:
def encode_qubit(bit, basis):
    qc = QuantumCircuit(1)
    if bit == 1:
        qc.x(0)
    if basis == "X":
        qc.h(0)
    return qc


def measure_qubit(qc, basis):
    measure_qc = qc.copy()

    if basis == "X":
        measure_qc.h(0)
    measure_qc.measure_all()
    return measure_qc

이제 BB84 프로토콜에서 Alice와 Bob의 역할을 하는 함수를 만들어봅시다.

Alice가 랜덤으로 비밀키를 생성하고, 랜덤으로 기저를 선택한 후 큐빗으로 인코딩하는 `alice()`은 완성돼있습니다.

❗️ `bob()`를 완성하세요. 이 함수는 Bob이 랜덤으로 기저를 선택한 후, Alice로 부터 받은 큐빗들을 `Sampler`을 사용해 해당 기저로 측정한 결과를 `bob_results` 리스트로 리턴합니다.

* 참고 : Bob의 측정결과를 `Sampler`로 실행할 때는, 한번만 측정해야하기 때문에, `shots=1`로 해야합니다

❗️ `eve()`을 완성하세요. 이 함수는 Eve가 Alice가 Bob에게 보내는 큐빗들을 도청하는 함수입니다.

이 함수는 Eve가 랜덤으로 기저를 선택 한 후, Alice로 부터 받은 큐빗들을 `Sampler`을 사용해 해당 기저로 측정합니다.

그리고 그 결과를 선택한 기저로 인코딩한 후`tampered_qubits`리스트에 넣어서 리턴합니다.

In [ ]:
bases = ["Z", "X"]


def alice(num_bits):
    alice_bits = np.random.randint(2, size=num_bits)
    alice_bases = np.random.choice(bases, size=num_bits)

    encoded_qubits = []
    for i in range(num_bits):
        encoded_qubits.append(encode_qubit(alice_bits[i], alice_bases[i]))

    return alice_bits, alice_bases, encoded_qubits


def bob(encoded_qubits, pm, sampler):
    num_bits = len(encoded_qubits)
    bob_bases = np.random.choice(bases, size=num_bits)

    bob_results = []
    for i in range(num_bits):
        measurement_circuit = measure_qubit(encoded_qubits[i], bob_bases[i])

        # TODO

        measured_bit = list(counts.keys())[0]
        bob_results.append(measured_bit)

    return bob_bases, bob_results


def eve(encoded_qubits, pm, sampler):
    num_bits = len(encoded_qubits)
    eve_bases = np.random.choice(bases, size=num_bits)

    tampered_qubits = []
    print("Eve가 중간에서 큐비트를 가로채는 중...")

    # TODO

    return tampered_qubits

`compare_bases()` 함수를 완성하세요. 

이 함수는 Alice와 Bob이 서로 공개된 채널을 통해 서로가 선택한 기저를 비교하고 같은 기저를 선택한 index를 선별하는 함수입니다.

In [ ]:
def compare_bases(alice_bases, bob_bases):
    same_bases_index = []

    # TODO

    return same_bases_index

`check_eve()` 함수를 완성하세요. 이 함수는 Eve가 있었는지 확인하는 함수입니다.

`mismatches`를 계산해 Alice와 Bob이 같은 기저를 선택한 비트의 오류율을 계산하는 부분을 완성하세요.

* 참고 : `alice_bits`와 `bob_bits`의 element들이 서로 다른 타입이니 `int()`로 int형으로 바꿔서 비교하세요.

In [ ]:
def check_eve(alice_bits, bob_bits, same_bases_index):
    sample_ratio = 0.5
    error_threshold = 0.01

    alice_sifted_key = [alice_bits[i] for i in same_bases_index]
    bob_sifted_key = [bob_bits[i] for i in same_bases_index]

    key_length = len(alice_sifted_key)
    if key_length == 0:
        print("🚨 기저가 일치하는 키가 없습니다.")
        return [], False

    sample_size = int(key_length * sample_ratio)
    alice_sample = alice_sifted_key[:sample_size]
    bob_sample = bob_sifted_key[:sample_size]

    mismatches = 0
    # TODO
    error_rate = mismatches / sample_size

    print(
        f"도청자 검사: {sample_size}개의 샘플 비트 비교 중 {mismatches}개 불일치 발견."
    )
    print(f"   - 오류율: {error_rate:.2%}")

    if error_rate > error_threshold:
        print("🚨 오류율이 높습니다! Eve가 도청했을 수 있습니다.")
        return None, True
    else:
        print("✅ 안전한 통신으로 확인되었습니다. 최종 키를 생성합니다.")
        return alice_sifted_key[sample_size:], False

In [ ]:
# Grader Cell

p16_grader = grader.grade_p16()
p16_grader.bb84_test(alice, bob, eve, compare_bases, check_eve)

---
#### 아래 셀을 실행해 최종 제출 해주세요.

In [ ]:
# Final Submission
grader.final_submission()


### 🎉🎉🎉 축하합니다! Beginner 해커톤을 완료하셨습니다! 🎉🎉🎉🎉

#### 자동으로 경품 추첨에 응모되셨습니다.

#### 조금더 어려운 문제에 도전해보고 싶으시면 Challenger Hackathon도 도전해보세요!

#### 수고하셨습니다 😃

#### &copy; 2025 Quantum Informatics at Yonsei Academy. All Rights Reserved

<details>
<summary style="font-size: large; font-weight: bold;">Credits</summary>

Notebook by Justin J. Kim based on materials by IBM Quantum

Grader by Justin J. Kim

Contact : j.hwankim@yonsei.ac.kr
</details>